In [25]:
!pip2 install numpy

#!python2 "/content/drive/My Drive/Research/Sent. Mining/mov.py"

In [26]:
#from google.colab import drive
#drive.mount('/content/drive/')
#!python2 '/content/drive/My Drive/Research/Sent.Mining/mov.py'
#with open('/content/drive/My Drive/Research/Sent.Mining/foo.txt', 'w') as f:
#  f.write('Hello Google Drive!')
#!cat /content/drive/My\ Drive/Research/Sent.Mining/foo.txt

Traceback (most recent call last):
  File "/content/drive/My Drive/Research/Sent.Mining/mov.py", line 103, in <module>
    for train_index, test_index in kf.split(corpus,labels):
  File "/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_split.py", line 324, in split
    X, y, groups = indexable(X, y, groups)
  File "/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py", line 229, in indexable
    check_consistent_length(*result)
  File "/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py", line 204, in check_consistent_length
    " samples: %r" % [int(l) for l in lengths])
ValueError: Found input variables with inconsistent numbers of samples: [2010, 2000]


In [11]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 08 17:03:30 2017

@author: Habibur Rahman
"""

import os
import numpy as np
#from google.colab import drive
from numpy import sort
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

h = .02  # step size in the mesh

def make_Corpus(root_dir):
    polarity_dirs = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]    
    corpus = []    
  
    for polarity_dir in polarity_dirs:
        reviews = [os.path.join(polarity_dir,f) for f in os.listdir(polarity_dir)]
        for review in reviews:
            doc_string = "";
            with open(review) as rev:
                for line in rev:
                    doc_string = doc_string + line
            if not corpus:
                corpus = [doc_string]
            else:
                corpus.append(doc_string)
    #print "Corpus\n",corpus
    return corpus

  
#drive.mount('/content/drive/')
  
#Create a dictionary of words with its frequency

root_dir = '/content/drive/My Drive/Research/Sent.Mining/txt_sentoken'
corpus = make_Corpus(root_dir)


#Prepare feature vectors per training mail and its labels
labels = np.zeros(2000);
labels[0:1000]=0;
labels[1000:2000]=1; 
      
kf = StratifiedKFold(n_splits=10)



names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Logistic Regression", #"Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    LogisticRegression(),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

scaler=MinMaxScaler(copy=True, feature_range=(0, 1))

#TSVD   start from here
svd=TruncatedSVD(n_components=200,n_iter=5,random_state=42)
#svd=NMF(n_components=50)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
#after normalize
vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf=True,stop_words='english')
corpus = vectorizer.fit_transform(corpus)

#fit corpus using tsvd
corpus=lsa.fit_transform(corpus)
corpus=scaler.fit_transform(corpus)

for name, model in zip(names, classifiers):
    total = 0
    totalMat = np.zeros((2,2));
    for train_index, test_index in kf.split(corpus,labels):
        y_train, y_test = labels[train_index], labels[test_index]
        X_train = [corpus[i] for i in train_index]
        X_test = [corpus[i] for i in test_index]
        Total_Train_data = len(X_train)
        Total_Test_data = len(X_test)
        model.fit(X_train, y_train)
        result = model.predict(X_test)
        totalMat = totalMat + confusion_matrix(y_test, result, labels=[0,1])
        total = total+sum(y_test==result)
        thresholds = sort(XGBClassifier().feature_importances_)
        for thresh in thresholds:
          # select features using threshold
          selection = SelectFromModel(AdaBoostClassifier(), threshold=thresh, prefit=True)
          select_X_train = selection.transform(X_train)
          # train model
          selection_model = XGBClassifier()
          selection_model.fit(select_X_train, y_train)
          # eval model
          select_X_test = selection.transform(X_test)
          y_pred = selection_model.predict(select_X_test)
          predictions = [round(value) for value in y_pred]
          accuracy = accuracy_score(y_test, predictions)
          print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    print "\n"+name+":\n";
    print "Confusion matrix:\n",totalMat    
    print "performance:", (total/2000.0)*100.0

XGBoostError: ignored

total 8
drwx------ 3 root root 4096 Nov  7 16:45 drive
drwxr-xr-x 2 root root 4096 Nov  5 19:58 sample_data
